## Installing needed packages

In [1]:
#conda install -c conda-forge BeautifulSoup4 --yes 
#!conda install -c conda-forge requests --yes 
#!conda install -c conda-forge lxml --yes 
#!conda install -c conda-forge html5lib --yes 

### Importing needed libraries

In [4]:
from bs4 import BeautifulSoup # for webscarping
import requests #to call the link
import numpy as np
import pandas as pd

#for maps
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

### Using Beautiful Soup to parse the link after reading it using requests

In [3]:
#!conda install -c conda-forge lxml --yes 
source =  requests.get ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup  = BeautifulSoup(source,'lxml')
#print (soup.prettify())

#### Converting all needed tables from the link in the dataframe

In [60]:
for table in soup.find_all('table',class_='wikitable sortable'):
    s = table
#print(type(s))
st =  str(s)
dfs =  pd.read_html(st)[0]
print(dfs)

    Postcode           Borough           Neighborhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
282      M8Z         Etobicoke              Mimico NW
283      M8Z         Etobicoke     The Queensway West
284      M8Z         Etobicoke  Royal York South West
285      M8Z         Etobicoke         South of Bloor
286      M9Z      Not assigned           Not assigned

[287 rows x 3 columns]


## Optional - directly using ipython to display 'table'

In [289]:
#!conda install -c conda-forge ipython --yes 
#from IPython.display import display_html
#display_html(st, raw=True)

### To directly read a table from the link - will be using this for the problem

In [7]:
## using df as the final dataframe for part 1 of the question

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
print(df)

    Postcode           Borough           Neighborhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
282      M8Z         Etobicoke              Mimico NW
283      M8Z         Etobicoke     The Queensway West
284      M8Z         Etobicoke  Royal York South West
285      M8Z         Etobicoke         South of Bloor
286      M9Z      Not assigned           Not assigned

[287 rows x 3 columns]


### Renaming the column Postcode to PostalCode to match with the requirement

In [9]:
df = df.rename(columns= {"Postcode": "PostalCode"})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Filtering out the 'Not assigned' Borough values

In [15]:
df_filtered = df[df['Borough']!='Not assigned']
df_filtered

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


#### First copying over the Borough to Neighborhood for Neighborhood values as 'Not assigned'. This will help if we have to aggregate neighborhood values in one row





In [20]:
df_copy = df_filtered.copy()
df_copy.loc[df_copy['Neighborhood'] == 'Not assigned','Neighborhood'] = df_copy['Borough']
#df_copy

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


#### To check if the column value was copied - first check the uncopied dataframe and then the updated i.e. df_copy

In [36]:
print('Rows where Neighborhood and borough are same is:\n ',df_filtered.loc[df_filtered['Neighborhood']== df_filtered['Borough']]) # returns empty as there's no same value
print('Rows where Neighborhood is Not assigned:\n',df_filtered.loc[df_filtered['Neighborhood']== 'Not assigned']) # returns one row where Neighborhood is Not assigned
print('Rows where Neighborhood is copied from Borough:\n',df_copy.loc[df_copy['Neighborhood']== df_copy['Borough']]) # returns the row where neighborhood value was updated

Rows where Neighborhood and borough are same is:
  Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []
Rows where Neighborhood is Not assigned:
   PostalCode       Borough  Neighborhood
7        M7A  Queen's Park  Not assigned
Rows where Neighborhood is copied from Borough:
   PostalCode       Borough  Neighborhood
7        M7A  Queen's Park  Queen's Park


#### Checking the rows where the Neighborhood values would need to be aggregated

In [58]:
df_cnt = df_copy.groupby(['PostalCode','Borough']).count()
print('Final data should have {} rows'.format(df_cnt.shape[0]))

Final data should have 103 rows


#### Creating the final dataset as per requirements

In [74]:
df_final =  df_copy.groupby(['PostalCode','Borough']).agg(Neighborhood = ('Neighborhood',', '.join)).reset_index()
print(df_final)

    PostalCode      Borough                                       Neighborhood
0          M1B  Scarborough                                     Rouge, Malvern
1          M1C  Scarborough             Highland Creek, Rouge Hill, Port Union
2          M1E  Scarborough                  Guildwood, Morningside, West Hill
3          M1G  Scarborough                                             Woburn
4          M1H  Scarborough                                          Cedarbrae
..         ...          ...                                                ...
98         M9N         York                                             Weston
99         M9P    Etobicoke                                          Westmount
100        M9R    Etobicoke  Kingsview Village, Martin Grove Gardens, Richv...
101        M9V    Etobicoke  Albion Gardens, Beaumond Heights, Humbergate, ...
102        M9W    Etobicoke                                          Northwest

[103 rows x 3 columns]


In [75]:
print('Final data has {} rows'.format(df_final.shape[0]))

Final data has 103 rows
